In [1]:
import boto3
import numpy as np
import pandas as pd
import os

In [2]:
%env AWS_PROFILE = dynamodbSobCap

env: AWS_PROFILE=dynamodbSobCap


In [3]:
s3 = boto3.resource("s3")
bucket = s3.Bucket('sobesice-capital-data')

# now bucket is "attached" the S3 bucket name "mybucket"
print(bucket)
# s3.Bucket(name='mybucket')

print(dir(bucket))
#show you all class method action you may perform

s3.Bucket(name='sobesice-capital-data')
['Acl', 'Cors', 'Lifecycle', 'LifecycleConfiguration', 'Logging', 'Notification', 'Object', 'Policy', 'RequestPayment', 'Tagging', 'Versioning', 'Website', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_name', 'copy', 'create', 'creation_date', 'delete', 'delete_objects', 'download_file', 'download_fileobj', 'get_available_subresources', 'load', 'meta', 'multipart_uploads', 'name', 'object_versions', 'objects', 'put_object', 'upload_file', 'upload_fileobj', 'wait_until_exists', 'wait_until_not_exists']


In [4]:
obj = bucket.Object("crypto-data/bitmex/quotes/BCHUSD/2020-11-17.csv").get()

In [7]:
obj = bucket.Object("crypto-data/bitmex/quotes/BCHUSD/2020-11-17.csv").get()

nms =  ['timestamp', 'bid', 'ask', 'bidsize', 'asksize']
    
initial_df = pd.read_csv(obj['Body'], header=None, names=nms, parse_dates=True)
initial_df.head()

,timestamp,bid,ask,bidsize,asksize
0,2020-11-17T11:23:23.487Z,251.65,251.7,891,3870
1,2020-11-17T11:23:24.225Z,251.65,251.7,891,3989
2,2020-11-17T11:23:24.977Z,251.65,251.7,891,3870
3,2020-11-17T11:23:25.504Z,251.65,251.7,891,3989
4,2020-11-17T11:23:26.026Z,251.65,251.7,891,3991


In [8]:
initial_df.index = pd.to_datetime(initial_df.timestamp)

In [ ]:
initial_df.price.rolling('60min').mean().plot()

In [26]:
def read_boto_obj_to_df(ticker, date, style):
    """
    Read raw csv to pandas DF
      - type quotes or trades
    """
    root_path = 'crypto-data/bitmex/'
    if style == 'quotes':
        nms = ['timestamp', 'bid', 'ask', 'bidsize', 'asksize']
    elif style == 'trans':
        nms = ['timestamp', 'ordertype', 'volume', 'price', 'ticktype']
    else:
        raise ValueError("Unknown data processing type. Supported types are 'quotes' and ' trans'.")

    path = os.path.join(root_path, style + '/', ticker + '/', date + '.csv')
    print(path)
    obj = bucket.Object(path).get()
    
    df = pd.read_csv(obj['Body'], header=None, names=nms, parse_dates=True)
    df.index = pd.to_datetime(df.timestamp)
    return df

In [28]:
read_boto_obj_to_df('BCHUSD', '2020-11-17', 'trans')

crypto-data/bitmex/trans/BCHUSD/2020-11-17.csv


,timestamp,ordertype,volume,price,ticktype
timestamp,,,,,
2020-11-17 11:24:00.281000+00:00,2020-11-17T11:24:00.281Z,Sell,1,251.65,ZeroMinusTick
2020-11-17 11:24:06.577000+00:00,2020-11-17T11:24:06.577Z,Buy,151,251.70,PlusTick
2020-11-17 11:24:06.577000+00:00,2020-11-17T11:24:06.577Z,Buy,1899,251.70,ZeroPlusTick
2020-11-17 11:24:09.091000+00:00,2020-11-17T11:24:09.091Z,Buy,350,251.85,PlusTick
2020-11-17 11:24:09.091000+00:00,2020-11-17T11:24:09.091Z,Buy,471,251.85,ZeroPlusTick
...,...,...,...,...,...
2020-11-17 23:57:43.348000+00:00,2020-11-17T23:57:43.348Z,Buy,335,256.45,PlusTick
2020-11-17 23:58:00.837000+00:00,2020-11-17T23:58:00.837Z,Sell,1,256.35,MinusTick
2020-11-17 23:58:44.956000+00:00,2020-11-17T23:58:44.956Z,Buy,335,256.45,PlusTick


In [70]:
read_boto_obj_to_df(ticker, date, style)

In [11]:
os.path.join(path = 'crypto-data/bitmex/quotes/' + tick)

'crypto-data/bitmex/quotes/quotes'

In [71]:
read_boto_obj_to_df(obj['Body'], 'quotes')

,timestamp,bid,ask,bidsize,asksize
0,2020-11-17T11:23:23.487Z,251.65,251.7,891,3870
1,2020-11-17T11:23:24.225Z,251.65,251.7,891,3989
2,2020-11-17T11:23:24.977Z,251.65,251.7,891,3870
3,2020-11-17T11:23:25.504Z,251.65,251.7,891,3989
4,2020-11-17T11:23:26.026Z,251.65,251.7,891,3991
...,...,...,...,...,...
60952,2020-11-17T23:59:58.418Z,256.55,256.6,5181,669
60953,2020-11-17T23:59:58.569Z,256.55,256.6,5181,335
60954,2020-11-17T23:59:58.575Z,256.55,256.6,6847,335
60955,2020-11-17T23:59:58.632Z,256.55,256.6,6847,385


In [ ]:
ticker = BCHUSD
'crypto-data/bitmex/quotes/' + ticker +

In [9]:
[ob.key for ob in bucket.objects.filter(Prefix='crypto-data/')]

['crypto-data/bitmex/quotes/BCHUSD/2020-11-17.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-18.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-19.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-20.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-21.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-22.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-23.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-24.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-25.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-26.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-27.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-28.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-29.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-11-30.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-12-01.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-12-02.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-12-03.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-12-04.csv',
 'crypto-data/bitmex/quotes/BCHUSD/2020-12-05.csv',
 'crypto-dat

In [81]:
list(bucket.objects.filter(Prefix='crypto-data/'))

[s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-17.csv'),
 s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-18.csv'),
 s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-19.csv'),
 s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-20.csv'),
 s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-21.csv'),
 s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-22.csv'),
 s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-23.csv'),
 s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-24.csv'),
 s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-25.csv'),
 

In [6]:
for my_bucket_object in bucket.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='sobesice-capital-data', key='Honza-backup-smartvue.zip')
s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-17.csv')
s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-18.csv')
s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-19.csv')
s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-20.csv')
s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-21.csv')
s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-22.csv')
s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-23.csv')
s3.ObjectSummary(bucket_name='sobesice-capital-data', key='crypto-data/bitmex/quotes/BCHUSD/2020-11-24.csv')
s3.ObjectSummary(bucket_name='sobesice-ca

In [14]:
bucket.objects.all()

s3.Bucket.objectsCollection(s3.Bucket(name='sobesice-capital-data'), s3.ObjectSummary)